In [1]:
import numpy as np     
import pandas as pd              
import matplotlib.pyplot as plt
from matplotlib import cm

# Clases propias
from SignalClases import OrgSignal
from SignalClases import FiltSignal
from SignalClases import SWSignal
from SignalClases import IndSW
from SignalClases import saveData
from ImgPlot import Plot_Signals
from Hilbert import HilbClass
from CrossDetClass import crossDetector
from timeDetClass import tDetClass
from recSigClass import recClass
from saveList import save

In [2]:
##  Cargar una muestra para prueba
foldNum = 0             # CBX
fileNum = 5             # FR - 94
srate = 10000           # tasa de muestreo 

In [ ]:
## Señal Original 

orgSignal, path, file = OrgSignal.getData(foldNum,fileNum)
# print(path)               # ruta de la carpeta 
# print(file)               # num. de archivo
# print(orgSignal.shape)      # dimension del arreglo     

In [ ]:
## Señal Filtrada   250 - 550 Hz
filtSig,filtPath = FiltSignal.getData(foldNum,fileNum)           # Señal Filtrada
print(filtPath)
print(filtSig.shape)


In [ ]:
SWSig = SWSignal.getData(foldNum,fileNum)               # Señal filtrada para detección de ondas lentas 
print(SWSig.shape)

In [ ]:
## Indice de SW
elct = 0
indSW,labelInd = IndSW.getData(foldNum,fileNum,elct)
indSize = indSW.shape
print(indSize)
print(labelInd)

print(indSW[0,0])        # primeros indices de swr detectados [fila, columna]
print(indSW[0,1])

In [ ]:
## Grafica de señales
szOrg = orgSignal.shape
timevec = np.arange(0,szOrg[1],1)/srate  
print(timevec.shape)
